In [5]:
import keras
from keras.preprocessing.image import ImageDataGenerator 

In [6]:
#Define the parameters/arguments for ImageDataGenerator class
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,rotation_range=180,zoom_range=0.2,horizontal_flip=True)

test_datagen=ImageDataGenerator(rescale=1./255)

In [7]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='CyfbVlhpb88HCt21fx4u1-RwsYc8kWTEnhyO199kN_R5',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'forestfiredetection-donotdelete-pr-u0rtatnl4rnifa'
object_key = 'ff.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [8]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [9]:
pwd

'/home/wsuser/work'

In [10]:
import os
filenames=os.listdir('/home/wsuser/work/Dataset/Dataset/train_set')

In [11]:
#Applying ImageDataGenerator functionality to trainset
x_train=train_datagen.flow_from_directory('/home/wsuser/work/Dataset/Dataset/train_set',target_size=(128,128),batch_size=32,class_mode='binary') 

Found 436 images belonging to 2 classes.


In [12]:
#Applying ImageDataGenerator functionality to testset
x_test=test_datagen.flow_from_directory('/home/wsuser/work/Dataset/Dataset/test_set',target_size=(128,128),batch_size=32,class_mode='binary')

Found 121 images belonging to 2 classes.


In [13]:
#import model building libraries

#To define Linear initialisation import Sequential
from keras.models import Sequential
#To add layers import Dense
from keras.layers import Dense
#To create Convolution kernel import Convolution2D
from keras.layers import Convolution2D
#import Maxpooling layer
from keras.layers import MaxPooling2D
#import flatten layer
from keras.layers import Flatten
import warnings
warnings.filterwarnings('ignore')

In [14]:
#initializing the model
model=Sequential()

In [15]:
#add convolutional layer
model.add(Convolution2D(32,(3,3),input_shape=(128,128,3),activation='relu'))
#add maxpooling layer
model.add(MaxPooling2D(pool_size=(2,2)))
#add flatten layer 
model.add(Flatten()) 

In [16]:
#add hidden layer
model.add(Dense(150,activation='relu'))
#add output layer
model.add(Dense(1,activation='sigmoid'))

In [17]:
#configure the learning process
model.compile(loss='binary_crossentropy',optimizer="adam",metrics=["accuracy"])

In [18]:
#Training the model
model.fit_generator(x_train,steps_per_epoch=14,epochs=10,validation_data=x_test,validation_steps=4) 

Epoch 1/10
14/14 [==============================] - 27s 2s/step - loss: 3.0777 - accuracy: 0.6170 - val_loss: 0.3781 - val_accuracy: 0.8430
Epoch 2/10
14/14 [==============================] - 26s 2s/step - loss: 0.4779 - accuracy: 0.8050 - val_loss: 0.1237 - val_accuracy: 0.9421
Epoch 3/10
14/14 [==============================] - 25s 2s/step - loss: 0.1986 - accuracy: 0.9243 - val_loss: 0.1076 - val_accuracy: 0.9587
Epoch 4/10
14/14 [==============================] - 26s 2s/step - loss: 0.2046 - accuracy: 0.9220 - val_loss: 0.0956 - val_accuracy: 0.9669
Epoch 5/10
14/14 [==============================] - 26s 2s/step - loss: 0.1917 - accuracy: 0.9151 - val_loss: 0.0901 - val_accuracy: 0.9752
Epoch 6/10
14/14 [==============================] - 25s 2s/step - loss: 0.1643 - accuracy: 0.9289 - val_loss: 0.1196 - val_accuracy: 0.9752
Epoch 7/10
14/14 [==============================] - 25s 2s/step - loss: 0.1747 - accuracy: 0.9220 - val_loss: 0.1053 - val_accuracy: 0.9752
Epoch 8/10
14/14 [==

In [19]:
model.save("forest1.h5") 

In [20]:
!tar -zcvf forest-fire-model_new.tgz forest1.h5

forest1.h5


In [21]:
ls

Dataset/  forest1.h5  forest-fire-model_new.tgz


In [22]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 11.1 MB/s eta 0:00:01


In [24]:
# Replace the credentials that you got from Watson Machine Learning service
from ibm_watson_machine_learning import APIClient
wml_credentials={
                "url":"https://us-south.ml.cloud.ibm.com",
                "apikey":"Z4BMC3kx1thzvwrAjgVHVNJ9ohlU8eHZyg5cRoJ2YqhY"
                }
client=APIClient(wml_credentials)

In [25]:
client=APIClient(wml_credentials)

In [26]:
def guid_from_space_name(client,space_name):
    space=client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id'])

In [27]:
space_uid = guid_from_space_name(client,'forest_fire_project')
print("Space UID = " + space_uid)

Space UID = 487ce4db-f474-498e-90d1-b595a6346c49


In [28]:
client.set.default_space(space_uid)

'SUCCESS'

In [24]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [29]:
software_spec_uid=client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_spec_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [30]:
model_details=client.repository.store_model(model='forest-fire-model_new.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"CNN",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid,
    client.repository.ModelMetaNames.TYPE:"tensorflow_2.7"}
                                           )
model_id=client.repository.get_model_uid(model_details) 

This method is deprecated, please use get_model_id()


In [31]:
model_id

'acbaf3b0-53b9-47b0-8146-943e2351a8ba'

In [32]:
client.repository.download(model_id,'my_model.tar.gz')

Successfully saved model content to file: 'my_model.tar.gz'


'/home/wsuser/work/my_model.tar.gz'